In [1]:
import torch
import numpy as np
import os
import pickle
import h5py
import pandas as pd
import datetime
import time
from collections import defaultdict
from tqdm import trange, tqdm


In [2]:
QRELS_PATH = '/home/catalinlup/MyWorkspace/MasterThesis/datasets/qrels/msmarco_psg.qrels.train.tsv'
qrels = pd.read_csv(QRELS_PATH, sep='\t', header=None).to_numpy()
qrels
relevant_docs_by_qid = defaultdict(list)
for row in qrels:
    qid = row[0]
    docid = row[2]
    relevant_docs_by_qid[qid].append(docid)
pickle.dump(dict(relevant_docs_by_qid), open('./negative_sample_batches/relevant_docs_by_qid.pickle', 'wb'))

In [2]:
BATCHES = []
for bi in trange(0, 809):

    BATCHES.append(pickle.load(open(f'./negative_sample_batches/sample_batch_{bi}.pickle', 'rb')))

100%|████████████████████████████████████████| 809/809 [00:03<00:00, 232.63it/s]


In [28]:
def fetch_vectors(dataset, ids) -> dict:
  # document id to index
  return dataset['vectors'][ids]

In [29]:
DOC_EMBEDDINGS_PATH = '/home/catalinlup/MyWorkspace/MasterThesis/datasets/h5_indices/faiss.msmarco-v1-passage.tct_colbert.h5'

In [4]:
FINAL_DATASET = defaultdict(lambda: defaultdict(list))
# with h5py.File(DOC_EMBEDDINGS_PATH, 'r') as doc_vecs:
for batch in tqdm(BATCHES):
    for query in batch.keys():
        if len(batch[query]['pos_docs']) == 0:
            continue
        
        FINAL_DATASET[query]['pos'] = list(map(lambda x: x[0], batch[query]['pos_docs']))
        FINAL_DATASET[query]['pos_scores'] = list(map(lambda x: x[1], batch[query]['pos_docs']))
        FINAL_DATASET[query]['neg'] = list(map(lambda x: x[0], batch[query]['neg_docs']))
        FINAL_DATASET[query]['neg_scores'] = list(map(lambda x: x[1], batch[query]['neg_docs']))

        

100%|████████████████████████████████████████| 809/809 [00:02<00:00, 269.94it/s]


In [5]:
pickle.dump(dict(FINAL_DATASET), open('./triples_dataset/final_dataset_sparse.pickle', 'wb'))

In [6]:
print(len(FINAL_DATASET.keys()))

502930
